# Enriching the dataset

> This notebook is only for showing purposes! The more optimized form is in the 0.1_oipeline_bike_stations.py

In [1]:
# imports
import pandas as pd
from data_preprocessing import decompress_pickle
from data_visualization import return_and_save_bike_station_map

In [2]:
# load data
art_br = pd.DataFrame(decompress_pickle("../data/interim/ArtificalRentals11.pbz2")).append(pd.DataFrame(decompress_pickle("../data/interim/ArtificalRentals12.pbz2")))

station_locs = decompress_pickle("../data/interim/Stations.pbz2")

In [18]:
return_and_save_bike_station_map(art_br, station_locs).save("../images/bike_rental_stations.html")


Both start and end stations are equal!


In [3]:
station_locs.head(3)

,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,INSTALLED,LOCKED,INSTALL_DATE,REMOVAL_DATE,TEMPORARY_INSTALL,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS,X,Y,SE_ANNO_CAD_DATA,OWNER
0,328739114,347,Connecticut Ave & McKinley St NW,31315,38.964544,-77.075135,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,3,16,393488.397567,144072.925481,NaN,DC
1,328739115,348,15th & L St NW,31276,38.903660,-77.034846,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,16,3,396977.478449,137312.167823,NaN,DC
2,328739116,349,17th & G St NW,31277,38.898300,-77.039732,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,25,4,396553.410741,136717.337924,NaN,DC


In [4]:
len(art_br["Start station number"].unique()) == len(art_br["End station number"].unique())

True

In [5]:
unique_stats = (pd.DataFrame({"Start station number": (art_br["Start station number"]
                                                       .unique())})
                  .merge(pd.DataFrame({"NO_OF_BIKES": (art_br.groupby(["Start station number"])
                                                             .count()["Bike number"])}),
                        on="Start station number", how="left")
                  .rename(columns={"Start station number": "TERMINAL_NUMBER"}))
unique_stats["color"] = pd.cut(unique_stats['NO_OF_BIKES'],
                               bins=3,
                               labels=['green', 'orange', 'red'])
station_loc_full = pd.merge(unique_stats, station_locs[["TERMINAL_NUMBER", "LONGITUDE", "LATITUDE", "ADDRESS"]], on = "TERMINAL_NUMBER")

In [6]:
cuts = pd.cut(unique_stats['NO_OF_BIKES'],
              bins=3,
              retbins=True,
              labels=['green', 'orange', 'red'])

In [7]:
print(f"Green is from {0} to {round(cuts[1][1])};\nOrange is from {round(cuts[1][1])} to {round(cuts[1][2])};\nRed is from {round(cuts[1][2])} to {round(cuts[1][3])}")

Green is from 0 to 35721;
Orange is from 35721 to 71442;
Red is from 71442 to 107162


# Lets create a map of all bike stations

In [13]:
import folium

In [16]:
m = folium.Map(location=[((station_loc_full.LATITUDE.min() + station_loc_full.LATITUDE.max())/2), ((station_loc_full.LONGITUDE.min() +  station_loc_full.LONGITUDE.max()) / 2)],
               zoom_start = 12,
               no_touch=True,
               zoom_control=False,
               control_scale=True)

In [17]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend of usage over two years</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>High usage (71442 to 107162)</li>
    <li><span style='background:orange;opacity:0.7;'></span>Medium usage (35721 to 71442)</li>
    <li><span style='background:green;opacity:0.7;'></span>Small usage (0 to 35721)</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

for i in range(0, len(station_loc_full)-1):
    folium.Marker([station_loc_full["LATITUDE"][i], station_loc_full["LONGITUDE"][i]], icon=folium.Icon(color=station_loc_full["color"][i], icon = "bicycle", prefix='fa')).add_to(m)

m.get_root().add_child(macro)